In [4]:
from Script_dataset.dataset_download import dataset_download
from Script_model.model import create_model

image_shape = (33, 33, 3)  # Taille d'entrée (33x33 pixels, 3 canaux pour la couleur)
n1 = 64  # Nombre de filtres pour la 1ère couche
f1 = (9, 9)  # Taille du filtre pour la 1ère couche
n2 = 32  # Nombre de filtres pour la 2e couche
f2 = (1, 1)  # Taille du filtre pour la 2e couche
n3 = 3  # Nombre de filtres pour la dernière couche
f3 = (5, 5)  # Taille du filtre pour la dernière couche

model = create_model(image_shape,n1,f1,n2,f2,n3,f3)

train_images, val_images, test_images = dataset_download()

ModuleNotFoundError: No module named 'kagglehub'